In [ ]:
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import seaborn as sns
import os
import glob
from sklearn.model_selection import train_test_split
import shutil


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
%cd 'gdrive/MyDrive/Colab Notebooks/Face Recognition/'

Mounted at /content/gdrive
/content/gdrive/MyDrive/Colab Notebooks/Face Recognition


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D , Flatten, GlobalAveragePooling2D
from tensorflow.keras.applications import InceptionV3
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

In [ ]:
TRAIN_PATH = 'Dataset/train/'
TEST_PATH = 'Dataset/test/'
IMG_SIZE = (218, 178)
BATCH_SIZE = 32
EPOCH_SIZE = 20


In [ ]:
df_train = pd.read_csv('Dataset/list_train_clean.csv')
df_test = pd.read_csv('Dataset/list_test_clean.csv')

#df_train

In [ ]:
df_train.loc[df_train['Male'] == 1, ['Male']] = 'Male'
df_train.loc[df_train['Male'] == 0, ['Male']] = 'Female'
df_train

,Filenames,Male
0,098446.jpg,Female
1,189077.jpg,Female
2,140986.jpg,Female
3,038818.jpg,Male
4,104595.jpg,Male
...,...,...
3995,093110.jpg,Female
3996,198880.jpg,Male
3997,013392.jpg,Male
3998,098793.jpg,Female


In [ ]:
df_test.loc[df_test['Male'] == 1, ['Male']] = 'Male'
df_test.loc[df_test['Male'] == 0, ['Male']] = 'Female'
df_test

,Filenames,Male
0,122315.jpg,Male
1,153759.jpg,Male
2,115290.jpg,Male
3,157437.jpg,Female
4,192358.jpg,Female
...,...,...
995,144965.jpg,Female
996,026421.jpg,Male
997,151442.jpg,Female
998,012915.jpg,Female


In [ ]:
df_train['Male'] = df_train['Male'].astype("string")
df_test['Male'] = df_test['Male'].astype("string")
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4000 entries, 0 to 3999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Filenames  4000 non-null   object
 1   Male       4000 non-null   string
dtypes: object(1), string(1)
memory usage: 62.6+ KB


In [ ]:
train_datagen = ImageDataGenerator(
    rotation_range=15,
    rescale=1./255,
    shear_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True,
    width_shift_range=0.1,
    height_shift_range=0.1
)

train_generator = train_datagen.flow_from_dataframe(
    df_train,
    TRAIN_PATH,
    x_col='Filenames',
    y_col='Male',
    target_size=IMG_SIZE,
    class_mode='binary',
    batch_size=BATCH_SIZE
)

Found 4000 validated image filenames belonging to 2 classes.


In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_dataframe(
    df_test,
    TEST_PATH,
    x_col='Filenames',
    y_col='Male',
    target_size=IMG_SIZE,
    class_mode='binary',
    batch_size=BATCH_SIZE
)

Found 1000 validated image filenames belonging to 2 classes.


In [ ]:
base_model = tf.keras.applications.vgg16.VGG16(weights='imagenet', include_top=False, input_shape= IMG_SIZE + (3,))

58889256/58889256 [==============================] - 1s 0us/step


In [ ]:
base_model.trainable = False
base_model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 218, 178, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 218, 178, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 218, 178, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 109, 89, 64)       0         
                                                                 
 block2_conv1 (Conv2D)       (None, 109, 89, 128)      73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 109, 89, 128)      147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 54, 44, 128)       0     

In [ ]:
model = Sequential()
model.add(base_model)
model.add(Flatten())
model.add(Dense(256,activation='relu'))
model.add(Dense(256,activation='relu'))
model.add(Dense(64,activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(2, activation='sigmoid'))

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 6, 5, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 15360)             0         
                                                                 
 dense (Dense)               (None, 256)               3932416   
                                                                 
 dense_1 (Dense)             (None, 256)               65792     
                                                                 
 dense_2 (Dense)             (None, 64)                16448     
                                                                 
 dense_3 (Dense)             (None, 32)                2080      
                                                                 
 dense_4 (Dense)             (None, 32)                1

In [ ]:
from keras.optimizers import Adam
adam = Adam(learning_rate=0.01)

In [ ]:
model.compile(optimizer=adam, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint

earlystop = EarlyStopping(patience=10)
learning_rate_reduction = ReduceLROnPlateau(monitor='val_accuracy',
                                            patience=4,
                                            verbose=1,
                                            factor=0.5,
                                            min_lr=0.01)
# Define a ModelCheckpoint callback to save the model weights during training
checkpoint_callback = ModelCheckpoint(filepath='Dataset/vgg16_weights_lr01.h5',
                                      save_best_only=True,
                                      save_weights_only=False)

callbacks = [checkpoint_callback, earlystop, learning_rate_reduction]


In [ ]:
%%time
history = model.fit(train_generator,
                    validation_data = test_generator,
                    validation_steps=len(df_test)//BATCH_SIZE,
                    steps_per_epoch=len(df_train)//BATCH_SIZE,
                    epochs=30,
                    verbose = 1,
                    callbacks=callbacks
                    )

Epoch 1/30
125/125 [==============================] - 2572s 20s/step - loss: 1.0256 - accuracy: 0.7362 - val_loss: 0.2728 - val_accuracy: 0.9012 - lr: 0.0100
Epoch 2/30
125/125 [==============================] - 2501s 20s/step - loss: 0.2927 - accuracy: 0.8800 - val_loss: 0.2405 - val_accuracy: 0.9173 - lr: 0.0100
Epoch 3/30
125/125 [==============================] - 2519s 20s/step - loss: 0.2434 - accuracy: 0.9020 - val_loss: 0.2093 - val_accuracy: 0.9204 - lr: 0.0100
Epoch 4/30
125/125 [==============================] - 2502s 20s/step - loss: 0.2278 - accuracy: 0.9068 - val_loss: 0.1948 - val_accuracy: 0.9194 - lr: 0.0100
Epoch 5/30
125/125 [==============================] - 2482s 20s/step - loss: 0.2160 - accuracy: 0.9115 - val_loss: 0.1957 - val_accuracy: 0.9294 - lr: 0.0100
Epoch 6/30
125/125 [==============================] - 2481s 20s/step - loss: 0.1899 - accuracy: 0.9222 - val_loss: 0.1777 - val_accuracy: 0.9335 - lr: 0.0100
Epoch 7/30
125/125 [==============================] 

In [ ]:
np.save('training_history_vgg16_lr01.npy', history.history)
model.save("model_finish_vgg16_lr01.h5")